In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [2]:
import requests
import re
import os
import pandas as pd
import time
import random
import eventlet #用来定时
eventlet.monkey_patch()#必须加这条代码

In [3]:
#http://www.behindthename.com/top/
#定义需要爬取的名字列表

namefile="H:\\97_Projects\\201906_Names\\names.xlsx"
df=pd.read_excel(namefile)
namelist1=list(df['Name'])
namelist=[]
for n in namelist1:
#     print(n)
    if type(n)==str: 
        namelist.append(n)
print('读取到',len(namelist),"个名字")


读取到 450 个名字


In [4]:

for name in namelist:

    #文件夹
    dirpath = r'H:\\97_Projects\\201906_Names\\'+str(name)

    if not os.path.isdir(dirpath):
        os.mkdir(dirpath)

Traceback (most recent call last):
  File "C:\Users\Zhitao\Anaconda3\lib\site-packages\eventlet\hubs\hub.py", line 461, in fire_timers
    timer()
  File "C:\Users\Zhitao\Anaconda3\lib\site-packages\eventlet\hubs\timer.py", line 59, in __call__
    cb(*args, **kw)
  File "C:\Users\Zhitao\Anaconda3\lib\site-packages\eventlet\semaphore.py", line 147, in _do_acquire
    waiter.switch()
greenlet.error: cannot switch to a different thread


In [5]:
#一个timeout函数 不好用
#参考https://cloud.tencent.com/developer/article/1043966
import signal, functools

def timeout(seconds, error_message="Timeout Error: the cmd 30s have not finished."):
    def decorated(func):
        result = ""
 
        def _handle_timeout(signum, frame):
            global result
            result = error_message
            raise TimeoutError(error_message)
 
        def wrapper(*args, **kwargs):
            global result
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
 
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
                return result
            return result
 
        return functools.wraps(func)(wrapper)
 
    return decorated
 

In [6]:
# @timeout(10)
def down(ulr):
    try:

        res = requests.get(url)
        if str(res.status_code)[0] == "4":
            print("未下载成功：", url)

    except Exception as e:
            print("未下载成功：", url)
    return res

In [7]:
dirpath = r'H:\\97_Projects\\201906_Names\\'+str(name)

In [8]:
dirpath='H:\\\\97_Projects\\\\201906_Names\\\\Abbey'

In [9]:
len(os.listdir(dirpath))

30

In [11]:
ncount=0

for name in namelist:
    print("开始下载",name)
    ncount+=1
    
    name='Zoey'
    #建立文件夹
    dirpath = r'H:\\97_Projects\\201906_Names\\'+str(name)
    if not os.path.isdir(dirpath):
        os.mkdir(dirpath)
        
    #是不是已经有图片了而且大于15张了
    if len(os.listdir(dirpath))>=10:
        print(name,"已经存在")
        continue
    print("已经下载",ncount,"//",len(namelist))
        
    #read url and download
    url_pre='http://image.baidu.com/search/index?tn=baiduimage&ps=1&ct=201326592&lm=-1&cl=2&nc=1&ie=utf-8&word='
    url_name_page=url_pre+name

    print('searching',url_name_page)
    
    send_headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
              "Connection":"keep-alive"}

    html = requests.get(url_name_page,headers=send_headers).text
    urls = re.findall(r'"objURL":"(.*?)"', html)

    if not os.path.isdir(dirpath):
        os.mkdir(dirpath)

    index = 1
    for url in urls:
        
        #随机暂停时间
        time.sleep(random.random())
        print("Downloading:", name,url)
#         res=down(url)
        #下载内容
        Is_timeout=1
        
        with eventlet.Timeout(10,False):  #超时设置
            
            try:
                res = requests.get(url)
                if str(res.status_code)[0] == "4":
                    print("未下载成功：", url)

            except Exception as e:
                    print("未下载成功：", url)
                    continue #下载失败这里要跳过而不是继续写入，导致重复了图片
            Is_timeout=0
        
        if Is_timeout==1: 
            print('下载超时')
            continue
        
        filename = os.path.join(dirpath, str(index) + ".jpg") 

        with open(filename, 'wb') as f: 
            print('writing')
            f.write(res.content) 
            index += 1

        continue

    print("下载结束，一共 %s 张图片" % index)
    print('count',ncount)

开始下载 Abbey
已经下载 1 // 450
searching http://image.baidu.com/search/index?tn=baiduimage&ps=1&ct=201326592&lm=-1&cl=2&nc=1&ie=utf-8&word=Zoey
Downloading: Zoey http://pic.feizl.com/upload/allimg/170614/22134I1H-6.jpg
下载超时
Downloading: Zoey http://x.imagefapusercontent.com/u/damien1888/6358754/726389999/zoe035BMB_278962012.jpg
下载超时
Downloading: Zoey http://photo4.zastatic.com/images/photo/6986/27941334/1294709878922_2.jpg
下载超时
Downloading: Zoey http://www.feizl.com/upload2007/allimg/170628/19560BR3-10.jpg
writing
Downloading: Zoey http://pic.feizl.com/upload/allimg/170704/243jj2p0mk2wxb.jpg
writing
Downloading: Zoey http://www.xzsofts.com/cscms/images/mtmt/Upload2010/78/czjsnylbqdbxnzzoey/08.jpg
下载超时
Downloading: Zoey http://pic.feizl.com/upload/allimg/170614/2252433036-2.jpg
writing
Downloading: Zoey http://img.china-ef.com/news/201601/14/2016011403014416.jpg
writing
Downloading: Zoey http://00.minipic.eastday.com/20170413/20170413151041_fe93e23f6b642a8621f6fe7b5554c223_11.jpeg
writing
Dow

# 不带挤时间的版本

In [ ]:
ncount=0

for name in namelist:
    print("开始下载",name)
    ncount+=1
    
    #建立文件夹
    dirpath = r'H:\\97_Projects\\201906_Names\\'+str(name)
    if not os.path.isdir(dirpath):
        os.mkdir(dirpath)
        
    #是不是已经有图片了而且大于15张了
    if len(os.listdir(dirpath))>=15:
        print(name,"已经存在")
        continue
    print("已经下载",ncount,"//",len(namelist))
        
    #读取url并且下载图片
    url_pre='http://image.baidu.com/search/index?tn=baiduimage&ps=1&ct=201326592&lm=-1&cl=2&nc=1&ie=utf-8&word='
    url_name_page=url_pre+name

    print('searching',url_name_page)
    
    send_headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
              "Connection":"keep-alive"}

    html = requests.get(url_name_page,headers=send_headers).text
    urls = re.findall(r'"objURL":"(.*?)"', html)

    if not os.path.isdir(dirpath):
        os.mkdir(dirpath)

    index = 1
    for url in urls:
        
        #随机暂停时间
        time.sleep(random.random())
        print("Downloading:", name,url)
         
#         res=down(url)

        #下载内容
        Is_timeout=1
        

        try:
            res = requests.get(url)
            if str(res.status_code)[0] == "4":
                print("未下载成功：", url)

        except Exception as e:
                print("未下载成功：", url)
                continue #下载失败这里要跳过而不是继续写入，导致重复了图片
        Is_timeout=0
        
#         if Is_timeout==1: 
#             print('下载超时')
#             continue
        
        filename = os.path.join(dirpath, str(index) + ".jpg") 

        with open(filename, 'wb') as f: 
            print('writing')
            f.write(res.content) 
            index += 1

        continue

    print("下载结束，一共 %s 张图片" % index)
    print('count',ncount)

In [ ]:
#带着threading的 未完成

import threading

ncount=0

for name in namelist:
    ncount+=1
    
    #建立文件夹
    dirpath = r'H:\\97_Projects\\201906_Names\\'+str(name)
    if not os.path.isdir(dirpath):
        os.mkdir(dirpath)
        
    #是不是已经有图片了而且大于15张了
    if len(os.listdir(dirpath))>=15:
        print(name,"已经存在")
        continue
    print("已经下载",ncount,"//",len(namelist))
        
    #读取url并且下载图片
    url_pre='http://image.baidu.com/search/index?tn=baiduimage&ps=1&ct=201326592&lm=-1&cl=2&nc=1&ie=utf-8&word='
    url_name_page=url_pre+name

    print('searching',url_name_page)
    
    send_headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
              "Connection":"keep-alive"}

    html = requests.get(url_name_page,headers=send_headers).text
    urls = re.findall(r'"objURL":"(.*?)"', html)

    if not os.path.isdir(dirpath):
        os.mkdir(dirpath)

    index = 1
    for url in urls:
        time.sleep(random.random())
        print("Downloading:", name,url)
         
#         res=down(url)
        try:
           
            res = requests.get(url)
            if str(res.status_code)[0] == "4":
                print("未下载成功：", url)

        except Exception as e:
                print("未下载成功：", url)

        filename = os.path.join(dirpath, str(index) + ".jpg") 

        with open(filename, 'wb') as f: 
            print('writing')
            f.write(res.content) 
            index += 1

        continue

    print("下载结束，一共 %s 张图片" % index)
    print('count',ncount)